In [7]:
import ujson
import re

from tqdm import tqdm_notebook
from glob import glob

from gensim.models.word2vec import Word2Vec

In [8]:
def read_tweets(pattern):
    
    tweets = []
    for path in glob(pattern):
        with open(path) as fh:
            for line in tqdm_notebook(fh):
                
                tweet = ujson.loads(line)
                
                text = re.sub(r"http\S+", "", tweet['body'])
                tokens = re.findall('[a-z0-9#@]+', text)
                
                tweets.append(tokens)
                
    return tweets

In [9]:
rural = read_tweets('../../data/geo-lt10k.json/part-0000*')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [15]:
urban = read_tweets('../../data/geo-gt1m.json/part-0000*')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [12]:
rural_model = Word2Vec(rural, workers=8)

In [16]:
urban_model = Word2Vec(urban, workers=8)

KeyboardInterrupt: 

In [14]:
rural_model.most_similar('earth')

[('planet', 0.8281048536300659),
 ('street', 0.6224478483200073),
 ('purpose', 0.6202324032783508),
 ('world', 0.5635701417922974),
 ('path', 0.5606052279472351),
 ('surface', 0.5601348876953125),
 ('land', 0.5583188533782959),
 ('moon', 0.5535959601402283),
 ('ocean', 0.541732668876648),
 ('ground', 0.534354567527771)]